In [92]:
import pandas as pd
from tqdm import tqdm
import os

champion control score unit is seconds per minute

In [93]:
'''
1.champion control score unit is seconds per minute
2.Made with the average of all games, there is no division between the pre, middle or late stages
'''

'\n1.champion control score unit is seconds per minute\n2.Made with the average of all games, there is no division between the pre, middle or late stages\n'

In [94]:
df_train_pre = pd.read_csv("../datasets/train_pre.csv")
df_train_mid = pd.read_csv("../datasets/train_mid.csv")
df_train_late = pd.read_csv("../datasets/train_late.csv")
df_train = pd.read_csv("../datasets/processed_data/train.csv")
championID = pd.read_csv("../datasets/championID.csv")

In [95]:
df_train

,gameId,gameDuration,teamVictory,team_100_gold,team_200_gold,Player_1_pick,Player_1_position,Player_1_time_game,Player_1_gold,Player_1_xp,...,Player_9_dmg_taken,Player_9_time_ccing,Player_10_pick,Player_10_position,Player_10_time_game,Player_10_gold,Player_10_xp,Player_10_dmg_dealt,Player_10_dmg_taken,Player_10_time_ccing
0,398776824,1837,100,65405,58915,516,TOP,1837,11898,15994,...,23943,8,117,UTILITY,1837,7847,10195,6120,24057,28
1,4363972028,1566,200,48274,55955,150,TOP,1566,11144,13246,...,13158,18,111,UTILITY,1566,7619,9623,4525,8963,41
2,3168495552,1572,200,48206,61837,50,TOP,1572,11770,13555,...,18734,18,235,UTILITY,1572,13997,12369,28012,13220,27
3,1314237530,1901,100,72646,64186,420,TOP,1901,16797,18448,...,30680,5,32,UTILITY,1901,8702,12848,15059,33510,63
4,1193729142,1485,200,36338,55458,98,TOP,1485,7108,10690,...,20530,2,350,UTILITY,1485,8617,11778,8615,6494,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85795,396286960,1893,100,70023,65105,19,TOP,1893,9030,12912,...,28660,21,25,UTILITY,1893,11601,12654,23098,22031,50
85796,5938889352,913,200,23224,34077,85,TOP,913,4673,6238,...,4680,0,111,UTILITY,913,4239,4487,3757,7548,28
85797,1194627781,1838,200,53520,63474,92,TOP,1838,10033,12691,...,16915,2,201,UTILITY,1838,9638,12985,11833,19067,39
85798,396535287,800,200,18019,30387,114,TOP,800,5504,6430,...,4253,0,117,UTILITY,800,4738,4012,2730,3028,5


In [99]:
def attack_defense_score(df):
    dfs = []
    for i in range(1,11):
        df_tmp = df.loc[:,['gameDuration','Player_'+str(i)+'_pick','Player_'+str(i)+'_dmg_dealt','Player_'+str(i)+'_dmg_taken']]
        #df_tmp['control_score'] = df_tmp.apply(lambda x: x['Player_'+str(i)+'_time_ccing'] / x['gameDuration'] * 60, axis=1)
        df_tmp['atk'] = df_tmp['Player_'+str(i)+'_dmg_dealt']/df['gameDuration']*60
        df_tmp['def'] = df_tmp['Player_'+str(i)+'_dmg_taken']/df['gameDuration']*60

        df_sum = df_tmp.groupby(['Player_'+str(i)+'_pick']).size().reset_index(name='sum'+str(i))
        df_atk = df_tmp.groupby(['Player_'+str(i)+'_pick'])['atk'].sum().reset_index(name='atk'+str(i))
        df_def = df_tmp.groupby(['Player_'+str(i)+'_pick'])['def'].sum().reset_index(name='def'+str(i))

        atk_def = df_sum.merge(df_atk, on = ['Player_'+str(i)+'_pick'])
        atk_def = atk_def.merge(df_def, on = ['Player_'+str(i)+'_pick'])

        atk_def.rename(columns = {"Player_"+str(i)+"_pick":'id'},inplace=True)
        dfs.append(atk_def)
    
    # return dfs

    ads = championID.merge(dfs[0],on='id',how = 'left')
    for i in range(1,10):
        cs = cs.merge(dfs[i],on='id',how = 'left')
    ads.fillna(0,inplace=True)
    
    ads['sum'] = ads['sum1']+ads['sum2']+ads['sum3']+ads['sum4']+ads['sum5']+ads['sum6']+ads['sum7']+ads['sum8']+ads['sum9']+ads['sum10'] 
    ads['atk'] = ads['atk1']+ads['atk2']+ads['atk3']+ads['atk4']+ads['atk5']+ads['atk6']+ads['atk7']+ads['atk8']+ads['atk9']+ads['atk10'] 
    ads['control_score'] = ads['control_sum']/ads['sum']
    championControlScore = cs.loc[:,['id','name','control_score']]
    championControlScore = championControlScore.round(4)
    return championControlScore
    

In [100]:
dfs = attack_defense_score(df_train_pre)
dfs[0]

,id,sum1,atk1,def1
0,1,14,4420.047585,4923.633245
1,2,155,80673.724038,105763.574596
2,3,4,962.307638,2103.182072
3,4,2,1082.665090,1054.254738
4,5,7,3315.692781,5505.348310
...,...,...,...,...
129,555,2,917.059722,1440.064464
130,777,313,165469.733062,193708.574250
131,875,168,89844.290986,125526.726759
132,876,75,37454.568348,48367.197935


In [96]:
def control_score(df):
    dfs = []
    for i in range(1,11):
        df_tmp = df.loc[:,['gameDuration','Player_'+str(i)+'_pick','Player_'+str(i)+'_time_ccing']]
        #df_tmp['control_score'] = df_tmp.apply(lambda x: x['Player_'+str(i)+'_time_ccing'] / x['gameDuration'] * 60, axis=1)
        df_tmp['control_score'] = df_tmp['Player_'+str(i)+'_time_ccing']/df['gameDuration']*60
        df_sum = df_tmp.groupby(['Player_'+str(i)+'_pick']).size().reset_index(name='sum'+str(i))
        df_con = df_tmp.groupby(['Player_'+str(i)+'_pick'])['control_score'].sum().reset_index(name='control_sum'+str(i))
        control = df_sum.merge(df_con, on = ['Player_'+str(i)+'_pick'])
        control.rename(columns = {"Player_"+str(i)+"_pick":'id'},inplace=True)
        dfs.append(control)
    
    # return dfs

    cs = championID.merge(dfs[0],on='id',how = 'left')
    for i in range(1,10):
        cs = cs.merge(dfs[i],on='id',how = 'left')
    cs.fillna(0,inplace=True)
    
    cs['sum'] = cs['sum1']+cs['sum2']+cs['sum3']+cs['sum4']+cs['sum5']+cs['sum6']+cs['sum7']+cs['sum8']+cs['sum9']+cs['sum10'] 
    cs['control_sum'] = cs['control_sum1']+cs['control_sum2']+cs['control_sum3']+cs['control_sum4']+cs['control_sum5']+cs['control_sum6']+cs['control_sum7']+cs['control_sum8']+cs['control_sum9']+cs['control_sum10'] 
    cs['control_score'] = cs['control_sum']/cs['sum']
    championControlScore = cs.loc[:,['id','name','control_score']]
    championControlScore = championControlScore.round(4)
    return championControlScore
    


In [97]:
championControlScore = control_score(df_train)
championControlScore

,id,name,control_score
0,1,Annie,0.8863
1,2,Olaf,0.5031
2,3,Galio,1.2263
3,4,Twisted Fate,1.3591
4,5,Xin Zhao,0.7885
...,...,...,...
155,777,Yone,0.7292
156,875,Sett,0.8599
157,876,Lillia,0.8315
158,887,Gwen,0.2268


In [98]:
championControlScore.to_csv("../Useful Features/5.championControlScore.csv", index = False)